In [2]:
import tensorflow as tf

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

Exception: URL fetch failure on https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1002)

(2570724321264, 2570724321264)

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

# Cargar los datos MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Crear conjuntos de datos a partir de los tensores
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

# Normalizar las imágenes dentro del conjunto de datos
def normalize_image(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.one_hot(label, depth=10)
    return image, label

train_dataset = train_dataset.map(normalize_image)
test_dataset = test_dataset.map(normalize_image)

# Definir el tamaño del lote y mezclar el conjunto de entrenamiento
batch_size = 128
train_dataset = train_dataset.shuffle(buffer_size=len(train_labels)).batch(batch_size)

# Definir el modelo de la red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entrenar el modelo utilizando el conjunto de datos
model.fit(train_dataset, epochs=5)